# Diffusionsgleichung (2D)

## Einführung

Unter der Annahme, dass die thermischen Eigenschaften konstant sind und wir nur von einer radiogenen Wärmequelle ausgehen, ist die Wärmeleitgleichung in 2-D gegeben durch:

$\begin{equation}
\frac{\partial{T}}{\partial{t}} = \kappa \left( \frac{\partial^2{T}}{\partial{x^2}} + \frac{\partial^2{T}}{\partial{y}^2}\right) + \frac{Q_{I^\textrm{C}}}{\rho_0 c_p},
\end{equation}$

wobei 

- $\rho_0$ die Referenzdichte [ kg/m<sup>3</sup> ], 
- $c_p$ die spezifische Wärmekapazität [ J/kg/K ], 
- $\kappa = k/\rho_0/c_p$ die thermische Diffusivität [ m<sup>2</sup>/s ] und 
- $Q$ die radiogene Wärmeproduktion pro Volumen [ W/m<sup>3</sup> ] ist. 

Die Gleichung beschreibt die Änderung der Temperatur mit der Zeit unter der Annahme, dass die Temperatur sich nur durch Diffusion ausbreitet. 

## Das Problem

Nehmen wir and, dass die Gleichung $(1)$ in einem 2D rechteckigen Gebiet mir der Seitenlänge $\left(L\right)$ 200 km und der Tiefe $\left(H\right)$ 100 km definiert sei (Abb. 1). Die Temperatur an der Oberfläche $\left(T_{\textrm{Surf}}\right)$ sei 0 °C und steigt in dem Gebiet linear mit einem thermischen Gradienten an. Als Randbedingungen der Temperatur an den Seiten nehmen wir *Neumann* Bedingungen an. In einer Tiefe von $z_a = H/2$ nehmen wir eine horizontal gelegene Anomalie mit einer Mächtigkeit $\left(h_a\right)$ von 10 km an, welcher sich über die gesammte Breite des Gebietes erstreckt  (z.B. einen Sill). Die Anomalie hat eine Anfangstemperatur $\left(T_a\right)$ von 1600 °C und eine volumetrische Wärmeproduktionsrate $\left(Q_a\right)$ von $2.7 \cdot 10^{-6}$.

<img src="./Figures/Exercise05b_1.png" alt="drawing" width="450"/> <br>
**Abb. 1.** Modelaufbau 

Mit der Zeit kühlt der Sill ab und heizt dabei das umgebene Gestein auf. Durch die zusätzliche Wärmeproduktionsrate des Sill steigt die Temperatur im weitern Verlauf wieder an bis sich ein thermisches Gleichgewicht einstellt. 

## Die Lösung

### Diskretisierung

Um das Problem numerisch lösen zu können, müssen wir unsere Modeldomaine in ein numerisches Gitter unterteilen. Dabei nehmen wir an, das die Temperatur und alle weiteren thermischen Eigenschaften auf den sogenannten *zentralen* Gitterpunkten (*Centroids*) definiert sind (siehe Abb. 2). Die Aufteilung entspricht der Aufteilung des [zeit-unabhängigen Problems](./04_2D_Diffusion_Stationary.ipynb).

Zur Lösung unseres Problems benutzen wir wieder die *Ghost Nodes* unseres Gitters, welche eine korrekte Einbindung der Randbedingungen ermöglichen. 

#### Gitter und Indizierung

<img src="./Figures/Exercise04_2.png" alt="drawing" width="450"/> <br>
**Abb. 2.** Versetztes Gitternetz. Die Temperatur ist auf den zenralen Gitterpunkten, den Centroids (rote Kreise) definiert. Die grauen Kreise außerhalb der Modelldomaine sind die Ghost Nodes. 

Das gegebene versetzte Gitter ermöglicht eine sogenannte *konservative* finite Differenzen Approximation, wobei angenommen wird, dass der Wärmefluss $q_{i,j} = -k \frac{\partial{T}}{\partial{x_{i,j}}}$ auf dem Mittelpunkt der Gitterlinien und die Temperatur im Zentrum einer Gitterzelle definiert ist (streng genommen ist die Wärmeleitfähigkeit dann auch auf den Gitterlinien definiert; da diese allerdings konstant ist, müssen wir das hier nicht berücksichtigen; bei variablen thermischen Parameter muss die Gleichung etwas anders diskreditiert werden; siehe [Dokumentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffTwoD)). Außerdem können wir durch die zentralen Temperaturgitterpunkte, in Verbindung mit den *Ghost Nodes* relativ einfach Randbedingungen einbinden, die die gleiche Fehlerordnung besitzen, wie die zentralen Differenzenquotienten im Inneren unseres Models. 

Bei der Indizierung unserer Gitterpunkte unterscheiden wir zwischen *lokalen* und *globalen* Indizes. Der lokale Index beschreibt die Position auf dem $i$,$j$-Gitter. Der globale Index, ist ein durchlaufender Index von 1 bis $nc_x \cdot nc_y$, wobei $nc_i$ die Anzahl der Centroids in die jeweilige Raumrichtung ist, und entspricht der Anzahl der Gleichungen, d.h. die Gesamtanzahl der inneren Gitterpunkte. Der globale Index wird auch beim Aufstellen der Koeffizientenmatrix für unser lineares Gleichungssystem verwendet. 

Für jeden zentralen Gitterpunkt, also für jede Gleichung, gibt ein sogenannter numerischer Stempel (*stencil*) die Position der Gitterpunkte an, welche für die jeweilige Gleichung gültig sind. Die Koeffizienten für diese Gitterpunkte sind dann jeweils ungleich null und alle anderen gleich null. Die Nomenklatur für den Stempel richtet sich häufig nach der eines Kompases, d.h. wir besitzen Punkte auf: Süden, Westen, Zentral, Osten, und Norden (siehe Abb. 2). Für jede Gleichung, ist der globale Index ($I$) jedes Punktes des Stempels gegeben durch die relative Position zum Zentral Punkt ($I^\textrm{C}$) des Stempels, d.h.: 

$\begin{equation}\begin{split}
I^\textrm{S} & = I^\textrm{C} - nc_x,\\
I^\textrm{W} & = I^\textrm{C} - 1,\\ 
I^\textrm{C} & = I, \\
I^\textrm{E} & = I^\textrm{C} + 1,\\
I^\textrm{N} & = I^\textrm{C} + nc_x,
\end{split}\end{equation}$

wobei $nc_x$ die Anzahl der horizontalen Centroids ist, $I^\textrm{C}$ ist der zentral Referenzpunkt, und $I^\textrm{S},I^\textrm{W},I^\textrm{E},I^\textrm{N}$ die Punkte im Süden, Westen, Osten, und Norden dazu.

#### Finite Differenzen Approximation

Nun können wir die partielle Differentialgleichung durch unsere finiten Differenzen approximieren. 

*Explizit (Forward Euler Verfahren)*
$\begin{equation}
T_{I^\textrm{C}}^{n+1} = T_{I^\textrm{C}}^n + \kappa \Delta{t} \left(\frac{T_{I^\textrm{W}}^{n} - 2 T_{I^\textrm{C}}^{n} + T_{I^\textrm{E}}^{n}}{\Delta{x^2}} \right) + \kappa \Delta{t} \left(\frac{T_{I^\textrm{S}}^{n} - 2 T_{I^\textrm{C}}^{n} + T_{I^\textrm{N}}^{n}}{\Delta{y^2}} \right) + \frac{Q_{I^\textrm{C}} \Delta{t}}{\rho_0 c_p},
\end{equation}$

wobei $n+1$ und $n$ jeweils der neue und der aktuell Zeitschritt ist. 

*Implizit (Backward Euler Verfahren)*
$\begin{equation}
-b T_{I^\textrm{S}}^{n+1} - a T_{I^\textrm{W}}^{n+1} + (2a + 2b + c) T_{I^\textrm{C}}^{n} - a T_{I^\textrm{E}}^{n+1} - b T_{I^\textrm{N}}^{n+1} = cT_{I^\textrm{C}}^{n} + \frac{Q_{I^\textrm{C}}}{\rho_0 c_p},
\end{equation}$

wobei 

$\begin{equation}\begin{split}
a & = \kappa / \Delta{x^2}, \\ 
b & = \kappa / \Delta{y^2}, \textrm{ und } \\ 
c & = 1 / \Delta{t}.
\end{split}\end{equation}$

Diese Gleichungen bilden ein fünfbändiges Gleichungssystem der Form:

$\begin{equation}
\mathbf{K} \cdot \bm{x} = \bm{b}
\end{equation}$

wobei $\mathbf{K}$ die Koeffizientenmatrix (mit fünf von Null verschiedenen Diagonalen) ist, $\bm{x}$ der unbekannte Lösungsvektor (die Temperaturen an den Zellzentren zum Zeitschritt $n+1$) und $\bm{b}$ die bekannte rechte Seite darstellt (die aktuelle Temperatur auf den Centroids).

**Allgemeine Lösung**

*Defektkorrektur* 

Ähnlich wie beim 1D-Problem kann das Gleichungssystem auf allgemeine Weise mithilfe der Defektkorrektur gelöst werden. Die Wärmeleitungsgleichung wird durch Einführung eines Residualterms $r$ umformuliert, der die Abweichung von der exakten Lösung quantifiziert und durch sukzessive Korrekturschritte iterativ reduziert werden kann, um die Genauigkeit zu verbessern. In impliziter Form kann das Residuum berechnet werden durch:

$\begin{equation}
\mathbf{K} \cdot \bm{x} - \bm{b} = \bm{r},
\end{equation}$

wobei $\bm{r}$ das Residuum ist.

Die Koeffizienten der Matrix sind dieselben wie oben hergeleitet (Gleichung (5)). Im Rahmen der Defekkorrektur kann die Koeffizientenmatrix, für ein 2D Problem mit konstanten thermischen Eigenschaften, mit Hilfe der eingebauten Funktion `AssembleMatrix2Dc()` aufgebaut werden. 

Gleichung (7) ergibt nach etwas Algebra den Korrekturterm der initialen Temperaturschätzung:

$\begin{equation}
\delta \bm{T} = -\mathbf{K}^{-1} \bm{r}^k
\end{equation}$

sowie zur aktualisierten Temperatur nach einem Iterationsschritt:

$\begin{equation}
\bm{T}^{k+1} = \bm{T}^k + \delta \bm{T}.
\end{equation}$

---

Innerhalb von `GeoModBox.jl` wird das Residuum $\bm{r}$ auf den Centroids berechnet, wobei das erweiterte Temperaturfeld des aktuellen Zeitschritts, das die Temperaturwerte der Ghost Nodes einschließt, als initiale Temperaturschätzung verwendet wird:

$\begin{equation}
\frac{\partial{T_{\textrm{ext},I}}}{\partial{t}} - \kappa \left( \frac{\partial^2{T_{\textrm{ext},I}}}{\partial{x}^2} + \frac{\partial^2{T_{\textrm{ext},I}}}{\partial{y}^2} \right) - \frac{Q_{I^\textrm{C}}}{\rho_0 c_p} = \bm{r}_{I},
\end{equation}$

wobei $I$ die Gleichungsnummer ist. Die Diskretisierung der Gleichung in Raum und Zeit mittels impliziter finiter Differenzen ergibt:

$\begin{equation}
\frac{T_{\textrm{ext},I^\textrm{C}}^{n+1}-T_{\textrm{ext},I^\textrm{C}}^{n}}{\Delta{t}} - \kappa
\left( \frac{T_{\textrm{ext},I^\textrm{W}}^{n+1} - 2 T_{\textrm{ext},I^\textrm{C}}^{n+1} + T_{\textrm{ext},I^\textrm{E}}^{n+1}}{\Delta{x}^2} + \frac{T_{\textrm{ext},I^\textrm{S}}^{n+1} - 2 T_{\textrm{ext},I^\textrm{C}}^{n+1} + T_{\textrm{ext},I^\textrm{N}}^{n+1}}{\Delta{y}^2}
\right) - \frac{Q_{I^\textrm{C}}}{\rho_0 c_p} = \bm{r}_{I},
\end{equation}$

wobei $I^\textrm{C}$ der globale, zentrale Referenzpunkt des Fünf-Punkte-Stempels auf dem erweiterten Temperaturfeld für die Centroids ist. Das Umformen von Gleichung (11) und das Einsetzen der Koeffizienten mittels Gleichung (5) ergibt:

$\begin{equation}
-b T_{\textrm{ext},I^\textrm{S}}^{n+1} - a T_{\textrm{ext},I^\textrm{W}}^{n+1} +
\left(2a + 2b + c \right) T_{\textrm{ext},I^\textrm{C}}^{n+1} -
a T_{\textrm{ext},I^\textrm{E}}^{n+1} - b T_{\textrm{ext},I^\textrm{N}}^{n+1} -
c T_{\textrm{ext},I^\textrm{C}}^n - \frac{Q_{I^\textrm{C}}}{\rho_0 c_p} =
r_{I},
\end{equation}$

was der Matrixform von Gleichung (7) entspricht, wobei $T_{\textrm{ext},I}^{n+1}$ der unbekannte Vektor $\bm{x}$ ist (wobei nur die Centroids verwendet werden), $-cT_{\textrm{ext},I}^n -\frac{Q_{I^\textrm{C}}}{\rho_0 c_p}$ der bekannte Vektor $\bm{b}$ ist und $-a$, $-b$ sowie $\left(2a+2b+c\right)$ die Koeffizienten der von Null verschiedenen Diagonalen der Koeffizientenmatrix $\bm{K}$ sind.

Das Residuum kann mit Hilfe der eingebauten Funktion `ComputeResiduals2Dc!()` berechnet werden. Mit dem Residualvektor $\bm{r}$ und der Koeffizientenmatrix $\bm{K}$ kann der Korrekturterm für die Temperatur über Gleichung (8) berechnet werden. Die Korrektur wird anschließend genutzt, um die initiale Temperaturschätzung zu aktualisieren (Gleichung (9)). Dies wird wiederholt, bis das Residuum als hinreichend klein angesehen wird.

**Spezielle Lösung** 

Wenn das Problem linear ist und die exakte Lösung innerhalb eines einzigen Iterationsschritts erreicht wird, reduziert sich das Gleichungssystem auf Gleichung (6). Damit kann das Gleichungssystem direkt über eine *linke Matrixdivision* gelöst werden:

$\begin{equation}
\bf{x} = \mathbf{K}^{-1} \bm{b}.
\end{equation}$

Die Koeffizientenmatrix bleibt dabei unverändert, auch für die gegebenen Randbedingungen. Allerdings muss die rechte Seite entsprechend aktualisiert werden (Setzen von $\bm{r}=0$ und Hinzufügen der bekannten Parameter zur rechten Seite der Gleichungen). Die spezielle Lösung der Gleichung mit Hilfe der links Matrixdivision wird in den eingebauten Funktionen `ForwardEuler2Dc!()` und `BackwardEuler2Dc!()` angewand. 

In dieser Übung können beide Lösungsverfahren angewand werden, wobei bei der generellen Lösung der kombinierte Solver mit eingebunden ist. Für mehr Informationen zur Lösung der PDG mit Hilfe verschiedenener finiten Differenzendiskretisierung und den entsprechenden Bedingungen, sowie Vor- und Nachteilen, überprüft bitte die entsprechende [Dokumentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffTwoD).

Um das Problem mit Hilfe von Julia lösen zu können, müssen wir erst die nötigen Module (```ExtendableSparse, Plots```) und Submodule (```GeoModBox.HeatEquation.TwoD```) definieren: 

In [ ]:
using ?
using Printf, LinearAlgebra

Nun definieren wir die Model ($L$,$H$) und physikalischen Parameter ($k$, $Q$), so wie das Diskretisierungsverfahren. Als Diskretisierungsverfahren können wir wählen, in dem wir die Variable FDSchema wie folgt definieren: 

- Speziellen Solver für ein lineares Problem (`explizit` oder `implizit`)
- Kombimierter, genereller Solver mit *Defektkorrektur* (`dc`)

In [ ]:
FDSchema    =:?
# Physikalischer Parameter ---------------------------------------------- #
P      = (
    L       =   ?,          #   Länge des Models    [m]
    H       =   ?,          #   Höhe des Models     [m]
    k       =   ?,              #   Thermische Konduktivität [W/m/K]
    cp      =   ?,           #   Wärmekapazität [J/kg/K]
    ρ       =   ?,           #   Dichte [kg/m^3]
    K0      =   ?,         #   Kelvin bei 0 C
    Q0      =   ?,              #   Hintergrund Waermeproduktionsrate;
)
P1      = (
    κ       =   ?,   #   Thermische Diffusivität [m^2/s]
    Tbot    =   ?,      #   Temperatur am unteren Rand  [ K ]
    Ttop    =   ?,         #   Temperatur am oberen Rand   [ K ]
    Tplume  =   ?,      #   Temperatur des Plumes [ K ]
    Xanoma  =   ?,            #   x-Koordinate des Zentrums der Anomaly
    Wanoma  =   ?,                #   Breite der Anomaly
    Hanoma  =   ?,               #   Hoehe der Anomaly
    Danoma  =   ?,            #   Tiefe des Zentrums der Anomaly
    Tanoma  =   ?,      #   Temperatur der Anomalie [ C ]
    Qanoma  =   ?              #   Waermeproduktionsrate pro Volumen [W/m^3]
)
P   = merge(P,P1)
# ----------------------------------------------------------------------- #

Als nächstes definieren wir die Anzahl der Gitterpunkte und die Gitterabstände:

In [ ]:
# Numerische Parameter -------------------------------------------------- #
NC      = (
    x       =   ?,             # Gitterpunkte in x-Richtung
    y       =   ?,             # Gitterpunkte in z-Richtung
)
Δ       = (
    x       = ?,
    y       = ?
)
# ----------------------------------------------------------------------- #

Mit deren Hilfe lässt sich das numerische Gitter bestimmen (siehe Abb. 2): 

In [ ]:
# Erstellung des Gitters ------------------------------------------------ #
x       = (
    c       =   LinRange(?),
)
y       = (
    c       =   LinRange(?),
)
# ----------------------------------------------------------------------- #

Da wir ein zeit-abhängiges Problem betrachten, müssen wir als nächstes die Parameter für die Zeit definieren. Beachte, für den Fall des expliziten FD-Verfahren, müssen wir das Diffusionsstabilitätskriterium berücksichtigen. Für mehr Informationen bezüglich der Bedingungen und der Vor- und Nachteile der Diskretisierungsverfahren der Wärmeleitgleichung in 2D siehe die entsprechende [Dokumentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffTwoD).

In [ ]:
# Zeit Parameter -------------------------------------------------------- #
T       =   (
    dn      =   25,             #   Inkremente der Graphischen Darstellung,
                                #   d.h. hier nur jeder 25 Zeitschritt
    year    =   365.25*3600*24, #   Sekunden pro Jahr    
    dtfac   =   ?,            #   Multiplikationsfaktor fuer dt
    Δ       =   [0.0],          #   Zeitschrittlaenge
    nt      =   [0]
)
T1      =   (
    tmax    =   60 * 1e6 * T.year,    #   Maximale Laufzeit des Models in s
    Δ       =   T.dtfac * (1.0 / (2.0 * P.κ *(1.0/Δ.x^2 + 1/Δ.y^2)))
)
T           =   merge(T,T1)

nt          =   floor(Int,T.tmax/T.Δ)  # Anzahl der Zeitschritte
time        =   zeros(nt)
# ----------------------------------------------------------------------- #

Zur Visualisierung der Ergebnisse als eine Animation in einer GIF Datei, müssen noch der Ort und der Name der Datei festgelegt werden. Der Parameter `save_fig` beschreibt wie immer, ob das Ergebnis als *GIF* gespeichert werden soll (1), oder nicht (0).  

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("05_2D_Sill_",FDSchema)
save_fig    =   1
# ----------------------------------------------------------------------- #

Als nächstes definieren wir die Anfangsbedingungen und die Datenarrays für unser Problem:

In [ ]:
# Anfangstemperaturfeld ------------------------------------------------- #
D       =   (
    Q           =   zeros(?),
    T           =   zeros(?),
    T0          =   zeros(?),
    T_ex        =   zeros(?),
    T_ex0       =   zeros(?),
    Tmax        =   zeros(?),
    Tprofile    =   zeros(NC.y,nt),
)
# Hintergrundfeld fuer Waermequellen
D.Q     .=  P.Q0
        
# Temperatur der Lithosphaere - linear zunehmenden mit der Tiefe
for i = 1:NC.x, j = 1:NC.y
    ?
end
@. D.T0                     =   D.T
D.T_ex[2:end-1,2:end-1]     .=  D.T

# Visualize initial condition ---
p = heatmap(?)

contour!(?)
if save_fig == 0
    display(p)
end
# ----------------------------------------------------------------------- #

### Randbedingungen

Die Temperatur auf den *Ghost Nodes* ist für *Dirichlet* und *Neumann* Randbedingungen genau so definiert, wie im [stationären 2-D Fall](./04_2D_Diffusion_Stationary.ipynb), bzw. den dazugehörigen [expliziten](./02_1D_Heat_explicit.ipynb) und [impliziten](./03_1D_Heat_implicit.ipynb) 1-D Versionen. Bei der Initialisierung der Randbedingungen nutzen wir die Hilfe des Tuples `BC`. Das Tuple definierte die Art der Randbedingung (`type`) und den dafür benötigten Wert (`val`). 

>Achtung: Die Solver sind so konzipiert, dass für die Randbedingungen Werte an jedem Centroid definiert sind, sprich die Randbedingung muss als ein 1D Array angegeben werden. 

In [ ]:
# Randbedingungen ------------------------------------------------------- #
BC      =   (type    = (W=:Neumann, E=:Neumann, N=:Dirichlet, S=:Dirichlet),
           val     = (W=zeros(?),E=zeros(?),N=?,S=?))
# ----------------------------------------------------------------------- #

### Gleichungssystem

Lasst uns für den impliziten Fall nun die Koeffizientenmatrix und rechte Seite initialisieren: 

In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   ?
K       =   ExtendableSparseMatrix(?,?)
rhs     =   zeros(?)
# --- Defektkorrektur (dc) ---
niter   =   10
ϵ       =   1e-25
R       =   zeros(NC...)
∂2T     =   (∂x2=zeros(NC.x, NC.y), ∂y2=zeros(NC.x, NC.y),
                ∂x20=zeros(NC.x, NC.y), ∂y20=zeros(NC.x, NC.y))
# Definiere das Diskretisierungsschema -
#   C   =   0   -> Implizit (default)
#   C   =   0.5 -> CNA
#   C   =   1.0 -> Explizit
C       =   0.0
# ----------------------------------------------------------------------- #

### Zeitschleife

Numerisch können wir nun in einer Zeitschleife die *PDG* auf unterschiedliche Art und Weise mit Hilfe der eingebauten Funktionen aus dem ```GeoModBox.HeatEquation.TwoD``` Submodul lösen.

Für den Fall des generellen Solvers mit Hilfe der Defektkorrektur müssen wir in einer Iterationsschleife (`1:niter`) erst das Residuum berechnen (`ComputeResiduals2Dc!()`), dann die Koeffizietenmatrix aufstellen (`AssembleMatrix2Ds()`), den Korreturterm δT und zum Schluss die Temperatur aktualisieren. Diese Iteration wird durchgeführt, bis das Residuum R einen bestimmten Wert ϵ unterschreitet. 

Für den Fall der spezielle Solver mit Hilfe der links Matrixdivison, verwenden wirh die eingebaute Funktion für konstante thermische Eigenschaften, `ForwardEuler2Dc!()` und `BackwardEuler2Dc!()`. 

In [ ]:
# Zeitschleife ---------------------------------------------------------- #
for n = 1:nt
    println(n)
    # Speicher das Temperaturprofil bei x = L/2
    @. D.Tprofile[:, n] = (D.T[convert(Int, NC.x/2), :] +
                           D.T[convert(Int, NC.x/2) + 1, :]) / 2
    # Track the maximum of the mid-depth horizontal temperature
    D.Tmax[n] = maximum((D.T[:, convert(Int, NC.y/2)] +
                          D.T[:, convert(Int, NC.y/2) + 1]) / 2)

    if n>1
        if FDSchema==:dc
            for iter = 1:niter
                # Evaluate residual
                ComputeResiduals2Dc!( ? )
                @printf("||R|| = %1.4e\n", norm(R)/length(R))
                norm(R)/length(R) < ϵ ? break : nothing
                # Assemble linear system
                K  = AssembleMatrix2Dc( ? )
                # Solve for temperature correction: Cholesky factorisation
                Kc = cholesky(K.cscmatrix)
                # Solve for temperature correction: Back substitutions
                δT = -(Kc\R[:])
                # Update temperature
                @. D.T += δT[Num.T]
            end
            D.T0    .= D.T
        elseif FDSchema==:explicit     
            ForwardEuler2Dc!( ? )        
        elseif FDSchema==:implicit
            BackwardEuler2Dc!( ? )
        end             
        time[n]     =   time[n-1] + T.Δ
    end
    
    if mod(n,T.dn) == 0 || n == 1 || n == nt
        p = heatmap(?)

        contour!(?)
        if save_fig == 1
            Plots.frame(anim)
        else
            display(p)                        
        end
    end
end

### Results

Zur besseren Darstellung und Analyse der Ergebnisse, stellen wir noch ein vertikales Profil der Temperatur gegenüber der Tiefe, so wie das Maxiumum eines horizontalen Profils der Temperatur gegenüber der Zeit in der jeweiligen Mitte der Domaine graphisch dar: 

In [ ]:
q = plot(D.Tprofile[:,1:T.dn:end].-P.K0,y.c./1e3,
        label="",xlabel="T_{x=L/2}",ylabel="Depth [km]",
        title="Temperature profile",
        layout=(1,2),subplot=1)

plot!(q,Time./T.year/1e6,D.Tmax.-P.K0,
        label="",xlabel="Time [My]",ylabel="T_{max} [°C]",
        subplot=2)

Nun müssen wir noch die Animation erstellen und speichern:

In [ ]:
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)        
    savefig(string("./Results/05_Sill_TProfile_Tmax_",FDSchema,".png"))
else
    display(q)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))